In [1]:
from google.colab import drive
drive.mount('/content/drive')


import pandas as pd
import numpy as np

!pip install --target=$nb_path python-igraph
!apt-get install --target=$nb_path libcairo2-dev libjpeg-dev libgif-dev
!pip install --target=$nb_path pycairo
!pip install --target=$nb_path python-igraph leidenalg cairocffi
!pip install --target=$nb_path --upgrade python-igraph

import igraph
import cairocffi
import matplotlib.pyplot as plt

Mounted at /content/drive
     |████████████████████████████████| 3.2 MB 2.8 MB/s 
  Created wheel for python-igraph: filename=python_igraph-0.9.8-py3-none-any.whl size=9070 sha256=86459c4407eb16e9b3cdd346df3494878349522c2ff2748c342ea767dc44dfa6
  Stored in directory: /root/.cache/pip/wheels/15/86/ef/b8bcdfbcb1c489771ad256c7cd1eb4971cdb7f3f670938b798
Successfully built python-igraph
E: Command line option --target= is not understood in combination with the other options
     |████████████████████████████████| 344 kB 2.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  ERROR: Failed building wheel for pycairo
Failed to build pycairo
ERROR: Could not build wheels for pycairo which use PEP 517 and cannot be installed directly
     |████████████████████████████████| 1.4 MB 2.1 MB/s 
     |████████████████████████████████| 88 kB 8.1 MB/s 
  Created wheel for cairocffi: filename=cairocffi-1.3.0-py3-none-any.

In [2]:
import ast
sentence_level_df = pd.read_csv('/content/drive/MyDrive/Northwell Pipelines/Youtube Pipeline/8_semantic_roles/srl_sentence_level_youtube.csv')
sentence_level_df
sentence_level_df['uid'] = sentence_level_df['uid'].astype(str)
sentence_level_df['task'] = sentence_level_df['task'].astype(str)
sentence_level_df['key'] = sentence_level_df['uid'] + '_' + sentence_level_df['task']
len(sentence_level_df['key'].unique())

for index, row in sentence_level_df.iterrows():
  sentence_level_df.at[index, 'ap_relations'] = ast.literal_eval(row['ap_relations'])
  sentence_level_df.at[index, 'pa_relations'] = ast.literal_eval(row['pa_relations'])
  sentence_level_df.at[index, 'semantic_relations'] = ast.literal_eval(row['semantic_relations'])

In [3]:
from igraph import *
from numpy import mean
from numpy import std

subjects_list = sentence_level_df['key'].unique().tolist()

srl_all_features = []

for i in range (0, len(subjects_list)):

  
  subject_relations = sentence_level_df.loc[sentence_level_df['key'] == subjects_list[i]]
  
  srl_graph_features = []
  srl_graph_features.append(subjects_list[i])
  srl_graph_features.append(subject_relations['uid'].unique().tolist()[0])
  srl_graph_features.append(subject_relations['task'].unique().tolist()[0])
  sentence_count = len(subject_relations['sentence_id'].unique())
  srl_graph_features.append(sentence_count)
  subject_relations['ap_relations']
  ap_matrix = {}
  for each_set in subject_relations['ap_relations']:
    for relation in each_set.keys():
      ap_matrix[relation] = ap_matrix.get(relation, 0) + each_set[relation]
  action_matrix = []
  for each_edge in ap_matrix.keys():
    each_relation = []
    each_relation.append(each_edge[0])
    each_relation.append(each_edge[1])
    each_relation.append(ap_matrix[each_edge])
    action_matrix.append(each_relation)
  pa_matrix = {}
  for each_set in subject_relations['pa_relations']:
    for relation in each_set.keys():
      pa_matrix[relation] = pa_matrix.get(relation, 0) + each_set[relation]
  predication_matrix = []
  for each_edge in pa_matrix.keys():
    each_relation = []
    each_relation.append(each_edge[0])
    each_relation.append(each_edge[1])
    each_relation.append(pa_matrix[each_edge])
    predication_matrix.append(each_relation)
  semantic_matrix = {}
  for each_set in subject_relations['semantic_relations']:
    for relation in each_set.keys():
      semantic_matrix[relation] = semantic_matrix.get(relation, 0) + each_set[relation]
  sem_matrix = []
  for each_edge in semantic_matrix.keys():
    each_relation = []
    each_relation.append(each_edge[0])
    each_relation.append(each_edge[1])
    each_relation.append(semantic_matrix[each_edge])
    sem_matrix.append(each_relation)
  g_action = igraph.Graph.TupleList(action_matrix, weights=True, directed=True)
  action_nn = g_action.vcount()
  action_ne = g_action.ecount()
  action_ad = igraph.mean(g_action.degree())
  action_awd = igraph.mean(g_action.strength(weights='weight'))
  action_gd = g_action.density()
  action_aspl = g_action.average_path_length()
  action_cc = g_action.transitivity_undirected()
  try:
    action_lq = len(g_action.largest_cliques()[0])
  except:
    action_lq = 0
  try:
    i_out = g_action.degree('i', mode='out', loops=True)
    i_in = g_action.degree('i', mode='in', loops=True)
    i_activity = i_out/(i_out + i_in)
  except:
    i_activity = 0
  clusters = g_action.clusters()
  try:
    lscc = clusters.giant()
    action_lscc = lscc.vcount()
  except:
    action_lscc = 0
  action_diameter = g_action.diameter()
  action_triangles = len(g_action.cliques(max=3, min=3))
  ###
  if action_nn > 2:
    random_cc_list = []
    random_aspl_list = []
    random_lscc_list = []
    for i in range (0, 1000): 
      g_rand = Graph.Erdos_Renyi(n=action_nn, m=action_ne, directed=True,loops=True)
      random_cc = g_rand.transitivity_undirected()
      random_aspl = g_rand.average_path_length()
      random_cc_list.append(random_cc)
      random_aspl_list.append(random_aspl)
      clusters = g_rand.clusters()
      largest_cluster = clusters.giant() 
      random_lscc = largest_cluster.vcount()
      random_lscc_list.append(random_lscc)  
    action_cc_z = (action_cc - mean(random_cc_list)) / std(random_cc_list)
    action_aspl_z = (action_aspl - mean(random_aspl_list)) / std(random_aspl_list)
    action_lscc_z = (action_lscc - mean(random_lscc_list)) / std(random_lscc_list)
  else:
    action_cc_z = np.nan
    action_aspl_z = np.nan
    action_lscc_z = np.nan
  ###
  srl_graph_features.extend([action_nn, action_ne, action_ad, action_awd, action_gd, action_aspl, action_cc, action_lq, i_activity, action_lscc, action_diameter, action_triangles,action_cc_z, action_aspl_z, action_lscc_z])

  g_predication = igraph.Graph.TupleList(predication_matrix, weights=True, directed=True)
  predication_nn = g_predication.vcount()
  predication_ne = g_predication.ecount()
  predication_ad = igraph.mean(g_predication.degree())
  predication_awd = igraph.mean(g_predication.strength(weights='weight'))
  predication_gd = g_predication.density()
  predication_aspl = g_predication.average_path_length()
  predication_cc = g_predication.transitivity_undirected()
  try:
    predication_lq = len(g_predication.largest_cliques()[0])
  except:
    predication_lq = 0
  clusters = g_predication.clusters()
  try:
    lscc = clusters.giant()
    predication_lscc = lscc.vcount()
  except:
    predication_lscc = 0
  predication_diameter = g_predication.diameter()
  predication_triangles = len(g_predication.cliques(max=3, min=3))
  ###
  if predication_nn > 2:
    random_cc_list = []
    random_aspl_list = []
    random_lscc_list = []
    for i in range (0, 1000): 
      g_rand = Graph.Erdos_Renyi(n=predication_nn, m=predication_ne, directed=True,loops=True)
      random_cc = g_rand.transitivity_undirected()
      random_aspl = g_rand.average_path_length()
      random_cc_list.append(random_cc)
      random_aspl_list.append(random_aspl)
      clusters = g_rand.clusters()
      largest_cluster = clusters.giant() 
      random_lscc = largest_cluster.vcount()
      random_lscc_list.append(random_lscc)  
    predication_cc_z = (predication_cc - mean(random_cc_list)) / std(random_cc_list)
    predication_aspl_z = (predication_aspl - mean(random_aspl_list)) / std(random_aspl_list)
    predication_lscc_z = (predication_lscc - mean(random_lscc_list)) / std(random_lscc_list)
  else:
    predication_cc_z = np.nan
    predication_aspl_z = np.nan
    predication_lscc_z = np.nan
  ###
  srl_graph_features.extend([predication_nn, predication_ne, predication_ad,predication_awd, predication_gd, predication_aspl, predication_cc, predication_lq, predication_lscc, predication_diameter, predication_triangles, predication_cc_z,predication_aspl_z,predication_lscc_z])

  g_semantic = igraph.Graph.TupleList(sem_matrix, weights=True, directed=True)
  ap_nn = g_semantic.vcount()
  ap_ne = g_semantic.ecount()
  ap_ad = igraph.mean(g_semantic.degree())
  ap_awd = igraph.mean(g_semantic.strength(weights='weight'))
  ap_gd = g_semantic.density()
  ap_aspl = g_semantic.average_path_length()
  ap_cc = g_semantic.transitivity_undirected()
  try:
    ap_lq = len(g_semantic.largest_cliques()[0])
  except:
    ap_lq = 0
  clusters = g_semantic.clusters()
  try:
    lscc = clusters.giant()
    ap_lscc = lscc.vcount()
  except:
    ap_lscc = 0
  ap_diameter = g_semantic.diameter()
  ap_triangles = len(g_semantic.cliques(max=3, min=3))
  ###
  if ap_nn > 2:
    random_cc_list = []
    random_aspl_list = []
    random_lscc_list = []
    for i in range (0, 1000): 
      g_rand = Graph.Erdos_Renyi(n=ap_nn, m=ap_ne, directed=True,loops=True)
      random_cc = g_rand.transitivity_undirected()
      random_aspl = g_rand.average_path_length()
      random_cc_list.append(random_cc)
      random_aspl_list.append(random_aspl)
      clusters = g_rand.clusters()
      largest_cluster = clusters.giant() 
      random_lscc = largest_cluster.vcount()
      random_lscc_list.append(random_lscc)  
    ap_cc_z = (ap_cc - mean(random_cc_list)) / std(random_cc_list)
    ap_aspl_z = (ap_aspl - mean(random_aspl_list)) / std(random_aspl_list)
    ap_lscc_z = (ap_lscc - mean(random_lscc_list)) / std(random_lscc_list)
  else:
    ap_cc_z = np.nan
    ap_aspl_z = np.nan
    ap_lscc_z = np.nan
  ###
  srl_graph_features.extend([ap_nn, ap_ne, ap_ad,ap_awd, ap_gd, ap_aspl, ap_cc, ap_lq, ap_lscc, ap_diameter, ap_triangles, ap_cc_z, ap_aspl_z,ap_lscc_z])

  srl_all_features.append(srl_graph_features)

srl_columns = ['key','uid','task', 'sentence_count', 'action_nn', 'action_ne', 'action_ad', 'action_awd', 'action_gd', 'action_aspl', 'action_cc', 'action_lq','i_activity','action_lscc','action_diameter','action_triangles','action_cc_z', 'action_aspl_z', 'action_lscc_z', 'predication_nn', 'predication_ne', 'predication_ad','predication_awd', 'predication_gd', 'predication_aspl', 'predication_cc', 'predication_lq','predication_lscc','predication_diameter','predication_triangles', 'predication_cc_z', 'predication_aspl_z', 'predication_lscc_z','ap_nn', 'ap_ne', 'ap_ad','ap_awd', 'ap_gd', 'ap_aspl', 'ap_cc', 'ap_lq', 'ap_lscc', 'ap_diameter','ap_triangles', 'ap_cc_z', 'ap_aspl_z', 'ap_lscc_z']
srl_df = pd.DataFrame(data = srl_all_features, columns=srl_columns)
srl_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: RuntimeWarning: directionality of edges is ignored for directed graphs at src/cliques/cliques.c:1132
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:79: RuntimeWarning: Edge directions are ignored for clique calculations at src/cliques/cliquer_wrapper.c:57
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:114: RuntimeWarning: directionality of edges is ignored for directed graphs at src/cliques/cliques.c:1132
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:124: RuntimeWarning: Edge directions are ignored for clique calculations at src/cliques/cliquer_wrapper.c:57
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:159: RuntimeWarning: directionality of edges is ignored for directed graphs at src/cliques/cliques.c:1132
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:169: RuntimeWarning: Edge directions are ignored for clique calculations at src/cliques/cliquer_wrap

,key,uid,task,sentence_count,action_nn,action_ne,action_ad,action_awd,action_gd,action_aspl,action_cc,action_lq,i_activity,action_lscc,action_diameter,action_triangles,action_cc_z,action_aspl_z,action_lscc_z,predication_nn,predication_ne,predication_ad,predication_awd,predication_gd,predication_aspl,predication_cc,predication_lq,predication_lscc,predication_diameter,predication_triangles,predication_cc_z,predication_aspl_z,predication_lscc_z,ap_nn,ap_ne,ap_ad,ap_awd,ap_gd,ap_aspl,ap_cc,ap_lq,ap_lscc,ap_diameter,ap_triangles,ap_cc_z,ap_aspl_z,ap_lscc_z
0,Y4_S9_journaling,Y4_S9,journaling,8,14,16,2.285714,3.428571,0.087912,1.482759,0.057692,3,0.750000,2,2,1,-0.665136,-1.279868,-0.368222,25,37,2.960000,3.920000,0.061667,1.027027,0.140625,3,1,2,6,0.622139,-3.383705,-1.507485,25,53,4.240000,5.840000,0.088333,1.664122,0.324627,4,4,3,29,3.695959,-4.435364,-3.184956
1,Y32_S38_journaling,Y32_S38,journaling,17,35,44,2.514286,2.914286,0.036975,1.349206,0.124260,3,0.000000,1,3,7,1.394128,-2.517541,-1.167107,84,132,3.142857,4.904762,0.018933,1.070922,0.055356,4,2,2,36,1.234705,-5.954954,-2.388511,84,176,4.190476,6.119048,0.025244,2.035211,0.148802,5,3,5,118,7.189189,-8.309880,-7.673469
2,Z1_H1_journaling,Z1_H1,journaling,17,41,43,2.097561,4.634146,0.026220,1.475000,0.003836,3,0.953488,2,2,1,-1.062701,-1.906504,-0.505550,55,113,4.109091,6.218182,0.038047,1.702830,0.199085,4,1,4,58,6.155766,-7.071880,-5.932385,55,156,5.672727,9.672727,0.052525,1.885343,0.202922,5,13,3,125,5.790476,-11.276194,-11.996126
3,Z4_H5_journaling,Z4_H5,journaling,9,23,23,2.000000,2.608696,0.045455,1.000000,0.000000,2,0.958333,1,1,0,-1.040235,-2.307220,-0.816822,30,49,3.266667,4.066667,0.056322,1.092593,0.123967,3,1,2,10,0.553072,-3.860945,-1.900926,30,72,4.800000,6.066667,0.082759,1.716000,0.211823,4,5,2,43,1.787916,-6.269044,-5.437402
4,Z16_H19_journaling,Z16_H19,journaling,11,7,7,2.000000,2.857143,0.166667,1.000000,0.000000,2,0.875000,1,1,0,NaN,-1.614895,-0.773277,17,19,2.235294,2.705882,0.069853,1.000000,0.109091,3,1,1,2,0.028074,-2.217518,-0.921997,17,26,3.058824,3.882353,0.095588,1.642857,0.230769,4,2,2,9,0.955116,-2.001437,-1.223905
5,Z16_H20_journaling,Z16_H20,journaling,15,27,40,2.962963,3.629630,0.056980,2.007752,0.127413,4,0.850000,4,4,11,0.596401,-1.969152,-0.808406,63,103,3.269841,3.650794,0.026370,1.142857,0.076271,4,1,3,21,1.293400,-5.703815,-2.505696,63,143,4.539683,5.206349,0.036610,2.266207,0.218254,5,9,5,110,8.589754,-7.820220,-6.874249
6,Z20_H24_journaling,Z20_H24,journaling,12,45,51,2.266667,3.866667,0.025758,1.444444,0.022605,4,0.955556,1,2,7,-0.660716,-2.206066,-1.013228,64,112,3.500000,4.343750,0.027778,1.125984,0.186944,4,1,2,42,6.750286,-7.061128,-3.163446,64,163,5.093750,7.062500,0.040427,2.113736,0.181818,5,14,4,112,6.399149,-10.050807,-9.670609
7,Y1_S1_journaling,Y1_S1,journaling,12,27,33,2.444444,3.185185,0.047009,1.787500,0.000000,2,0.850000,1,3,0,-1.489782,-1.628986,-1.099803,71,103,2.901408,3.690141,0.020724,1.000000,0.068662,4,1,1,26,1.513165,-4.549258,-1.782548,71,136,3.830986,4.901408,0.027364,1.881215,0.143639,5,5,4,70,5.398681,-7.205291,-4.698793
8,Y5_S10_journaling,Y5_S10,journaling,30,21,22,2.095238,2.666667,0.052381,1.706897,0.023622,3,0.875000,2,3,1,-0.812236,-1.060334,-0.337831,41,64,3.121951,4.243902,0.039024,1.000000,0.044554,3,1,1,6,-0.857325,-4.345176,-1.828495,41,86,4.195122,5.609756,0.052439,1.925714,0.185874,4,2,3,50,3.213999,-5.998391,-5.231073
9,Y9_S14_journaling,Y9_S14,journaling,29,31,38,2.451613,4.129032,0.040860,1.588889,0.043860,3,0.896552,3,2,5,-0.580021,-2.017296,-0.497066,51,107,4.196078,6.823529,0.041961,1.268116,0.174006,4,1,3,54,4.226292,-8.321618,-6.110831,51,145,5.686275,9.333333,0.056863,2.056757,0.245183,5,12,4,123,7.398523,-9.485898,-11.628957


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Northwell Pipelines/Youtube Pipeline/7_filtered_words/filtered_words_youtube.csv')
df = df[df['sp.pos'].notna()].copy()
df_open_tokens =  df[['uid', 'task','sentence_id', 'token']].copy()
included_data = df[['uid','task', 'sentence_id', 'sp.lemma', 'sp.pos', 'token']].copy()

included_data['uid'] = included_data['uid'].astype(str)
included_data['task'] = included_data['task'].astype(str)
included_data['key'] = included_data['uid'] + '_' + included_data['task']
len(included_data['key'].unique())

48

In [5]:
#making a list of all word-level data, separating multiple items in each cell

all_words_data = []
for index, row in included_data.iterrows():
  words_data_extractor = []
  words_data_extractor.append(row['key'])
  words_data_extractor.append(row['uid'])
  words_data_extractor.append(row['task'])
  words_data_extractor.append(row['sentence_id'])
  words_data_extractor.append(row['sp.lemma'])
  words_data_extractor.append(row['sp.pos'])
  words_data_extractor.append(row['token'])
  #print(words_data_extractor)
  all_words_data.append(words_data_extractor)
 
print('number of rows:',len(all_words_data))

#all_words_data includes user_id, sentence_id, lemmatized and non-lemmatized forms of words, and parts of speech

#Removing excluded parts of speech

excluded_pos_data = []

included_pos = ['VERB', 'NOUN', 'ADV','ADJ','PRON','PROPN']

for each in all_words_data:
  if each[5] in included_pos:
    excluded_pos_data.append(each)
print('number of rows (after exclusion):', len(excluded_pos_data))

number of rows: 10226
number of rows (after exclusion): 6080


In [6]:
#Lemmatized graph features

all_lemmas = []
cooc_seq_list = []

for each in all_words_data:
  binder = []
  binder.append(each[0])
  tupple_id = (each[1], each[3])
  binder.append(each[1])
  binder.append(tupple_id)
  binder.append(each[2])
  if each[4] == '-PRON-':
    binder.append(each[6])
  else:
    binder.append(each[4])
  binder.append(each[5])
  all_lemmas.append(binder)


included_pos = ['VERB', 'NOUN', 'ADV','ADJ','PRON','PROPN']

#creating a dataframe from lemma data
labels = ['key', 'subject', 'subject-sentence','task', 'lemma','pos']
df_lm = pd.DataFrame.from_records(all_lemmas, columns=labels)
df_lm


subjects = df_lm['key'].unique().tolist()


for each in subjects:
  subject_df = df_lm.loc[df_lm['key'] == each]

  sentence_count = len(subject_df['subject-sentence'].unique())
  word_count = len(subject_df.index)
  subject_df_pos = subject_df.loc[subject_df.pos.isin(included_pos)]

  ###sequence graph
  seq_relations_matrix = {}
  sequential_list = subject_df_pos['lemma'].tolist()
  for i in range (0, len(sequential_list) - 1):
    a = (sequential_list[i], sequential_list[i+1])
    seq_relations_matrix[a] = seq_relations_matrix.get(a, 0) + 1
  seq_matrix = []
  for each_edge in seq_relations_matrix.keys():
    each_relation = []
    each_relation.append(each_edge[0])
    each_relation.append(each_edge[1])
    each_relation.append(seq_relations_matrix[each_edge])
    seq_matrix.append(each_relation)
  g_seq = igraph.Graph.TupleList(seq_matrix, weights=True, directed=True)
  seq_nn = g_seq.vcount()
  seq_ne = g_seq.ecount()
  seq_ad = igraph.mean(g_seq.degree())
  seq_awd = igraph.mean(g_seq.strength(weights='weight'))
  seq_gd = g_seq.density()
  seq_aspl = g_seq.average_path_length()
  seq_cc = g_seq.transitivity_undirected()
  try:
    seq_lq = len(g_seq.largest_cliques()[0])
  except:
    seq_lq = 0
    print(seq_nn)
  clusters = g_seq.clusters()
  try:
    lscc = clusters.giant()
    seq_lscc = lscc.vcount()
  except:
    seq_lscc = 0
  seq_lscc = lscc.vcount()
  seq_diameter = g_seq.diameter()
  seq_triangles = len(g_seq.cliques(max=3, min=3))
    ###
  if seq_nn > 2:
    random_cc_list = []
    random_aspl_list = []
    random_lscc_list = []
    for i in range (0, 1000): 
      g_rand = Graph.Erdos_Renyi(n=seq_nn, m=seq_ne, directed=True,loops=True)
      random_cc = g_rand.transitivity_undirected()
      random_aspl = g_rand.average_path_length()
      random_cc_list.append(random_cc)
      random_aspl_list.append(random_aspl)
      clusters = g_rand.clusters()
      largest_cluster = clusters.giant() 
      random_lscc = largest_cluster.vcount()
      random_lscc_list.append(random_lscc)  
    seq_cc_z = (seq_cc - mean(random_cc_list)) / std(random_cc_list)
    seq_aspl_z = (seq_aspl - mean(random_aspl_list)) / std(random_aspl_list)
    seq_lscc_z = (seq_lscc - mean(random_lscc_list)) / std(random_lscc_list)
  else:
    seq_cc_z = np.nan
    seq_aspl_z = np.nan
    seq_lscc_z = np.nan
  ###
  ### co-occurrence graph
  coc_relations_matrix = {}
  subject_sentence = subject_df_pos['subject-sentence'].unique().tolist()
  for each_sentence in subject_sentence:
    word_list = subject_df_pos.loc[subject_df_pos['subject-sentence'] == each_sentence]['lemma'].tolist()
    for first_element in range(0, len(word_list)-1):
      for second_element in range(first_element + 1, len(word_list)):     
        a = tuple(sorted([word_list[first_element], word_list[second_element]]))
        coc_relations_matrix[a] = coc_relations_matrix.get(a, 0) + 1
  coc_matrix = []
  for each_edge in coc_relations_matrix.keys():
    each_relation = []
    each_relation.append(each_edge[0])
    each_relation.append(each_edge[1])
    each_relation.append(coc_relations_matrix[each_edge])
    coc_matrix.append(each_relation)
  g_cooc = igraph.Graph.TupleList(coc_matrix, weights=True, directed=False)
  cooc_nn = g_cooc.vcount()
  cooc_ne = g_cooc.ecount()
  cooc_ad = igraph.mean(g_cooc.degree())
  cooc_awd = igraph.mean(g_cooc.strength(weights='weight'))
  cooc_gd = g_cooc.density()
  cooc_aspl = g_cooc.average_path_length()
  cooc_cc = g_cooc.transitivity_undirected()
  try:
    cooc_lq = len(g_cooc.largest_cliques()[0])
  except:
    cooc_lq = 0
    print(cooc_nn)
  cooc_diameter = g_cooc.diameter()

  coc_seq_features = []
  coc_seq_features.append(each)
  coc_seq_features.append(sentence_count)
  coc_seq_features.append(word_count)
  mean_sentence_length = word_count/sentence_count
  coc_seq_features.append(mean_sentence_length)
  coc_seq_features.append(seq_nn)
  coc_seq_features.append(seq_ne)
  coc_seq_features.append(seq_ad)
  coc_seq_features.append(seq_awd)
  coc_seq_features.append(seq_gd)
  coc_seq_features.append(seq_aspl)
  coc_seq_features.append(seq_cc)
  coc_seq_features.append(seq_lq)
  coc_seq_features.append(seq_lscc)
  coc_seq_features.append(seq_diameter)
  coc_seq_features.append(seq_triangles)
  coc_seq_features.append(seq_cc_z)
  coc_seq_features.append(seq_aspl_z)
  coc_seq_features.append(seq_lscc_z)
  coc_seq_features.append(cooc_nn)
  coc_seq_features.append(cooc_ne)
  coc_seq_features.append(cooc_ad)
  coc_seq_features.append(cooc_awd)
  coc_seq_features.append(cooc_gd)
  coc_seq_features.append(cooc_aspl)
  coc_seq_features.append(cooc_cc)
  coc_seq_features.append(cooc_lq)
  coc_seq_features.append(cooc_diameter) 
  cooc_seq_list.append(coc_seq_features)

cooc_seq_df = pd.DataFrame(data=cooc_seq_list, columns=['key', 'sentence_count', 'word_count', 'mean_sentence_length','seq_nn', 'seq_ne', 'seq_ad', 'seq_awd', 'seq_gd', 'seq_aspl', 'seq_cc', 'seq_lq', 'seq_lscc', 'seq_diameter','seq_triangles','seq_cc_z','seq_aspl_z','seq_lscc_z','cooc_nn', 'cooc_ne', 'cooc_ad', 'cooc_awd', 'cooc_gd', 'cooc_aspl', 'cooc_cc', 'cooc_lq', 'cooc_diameter'])
cooc_seq_df.head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: RuntimeWarning: directionality of edges is ignored for directed graphs at src/cliques/cliques.c:1132
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: RuntimeWarning: Edge directions are ignored for clique calculations at src/cliques/cliquer_wrapper.c:57


,key,sentence_count,word_count,mean_sentence_length,seq_nn,seq_ne,seq_ad,seq_awd,seq_gd,seq_aspl,seq_cc,seq_lq,seq_lscc,seq_diameter,seq_triangles,seq_cc_z,seq_aspl_z,seq_lscc_z,cooc_nn,cooc_ne,cooc_ad,cooc_awd,cooc_gd,cooc_aspl,cooc_cc,cooc_lq,cooc_diameter
0,Y4_S9_journaling,8,90,11.250000,29,45,3.103448,3.517241,0.055419,4.230867,0.125749,3,28,10,7,0.642822,1.071884,3.618408,28,131,9.357143,18.714286,0.346561,1.556738,0.764110,13,2
1,Y32_S38_journaling,17,248,14.588235,98,131,2.673469,2.755102,0.013781,9.351047,0.010753,3,97,26,1,-1.049462,3.056280,7.437749,98,655,13.367347,16.448980,0.137808,2.475910,0.757773,23,5
2,Z1_H1_journaling,17,219,12.882353,66,113,3.424242,4.090909,0.026340,4.328284,0.072753,4,65,11,17,1.154894,-1.513560,3.675586,65,363,11.169231,21.630769,0.174519,1.901740,0.510935,13,3
3,Z4_H5_journaling,9,95,10.555556,39,53,2.717949,2.871795,0.035762,4.616915,0.051724,3,37,11,4,-0.332593,1.158426,5.515412,39,211,10.820513,14.358974,0.284750,1.716599,0.768823,15,2
4,Z16_H19_journaling,11,59,5.363636,30,39,2.600000,2.666667,0.044828,5.158430,0.066667,3,23,13,2,-0.253959,2.715686,4.335952,29,66,4.551724,5.310345,0.162562,1.615385,0.612903,7,2


In [7]:
integrated_graphs = srl_df.merge(cooc_seq_df.drop('sentence_count', axis=1), on='key')
integrated_graphs.head(5)

,key,uid,task,sentence_count,action_nn,action_ne,action_ad,action_awd,action_gd,action_aspl,action_cc,action_lq,i_activity,action_lscc,action_diameter,action_triangles,action_cc_z,action_aspl_z,action_lscc_z,predication_nn,predication_ne,predication_ad,predication_awd,predication_gd,predication_aspl,predication_cc,predication_lq,predication_lscc,predication_diameter,predication_triangles,predication_cc_z,predication_aspl_z,predication_lscc_z,ap_nn,ap_ne,ap_ad,ap_awd,ap_gd,ap_aspl,ap_cc,ap_lq,ap_lscc,ap_diameter,ap_triangles,ap_cc_z,ap_aspl_z,ap_lscc_z,word_count,mean_sentence_length,seq_nn,seq_ne,seq_ad,seq_awd,seq_gd,seq_aspl,seq_cc,seq_lq,seq_lscc,seq_diameter,seq_triangles,seq_cc_z,seq_aspl_z,seq_lscc_z,cooc_nn,cooc_ne,cooc_ad,cooc_awd,cooc_gd,cooc_aspl,cooc_cc,cooc_lq,cooc_diameter
0,Y4_S9_journaling,Y4_S9,journaling,8,14,16,2.285714,3.428571,0.087912,1.482759,0.057692,3,0.750000,2,2,1,-0.665136,-1.279868,-0.368222,25,37,2.960000,3.920000,0.061667,1.027027,0.140625,3,1,2,6,0.622139,-3.383705,-1.507485,25,53,4.240000,5.840000,0.088333,1.664122,0.324627,4,4,3,29,3.695959,-4.435364,-3.184956,90,11.250000,29,45,3.103448,3.517241,0.055419,4.230867,0.125749,3,28,10,7,0.642822,1.071884,3.618408,28,131,9.357143,18.714286,0.346561,1.556738,0.764110,13,2
1,Y32_S38_journaling,Y32_S38,journaling,17,35,44,2.514286,2.914286,0.036975,1.349206,0.124260,3,0.000000,1,3,7,1.394128,-2.517541,-1.167107,84,132,3.142857,4.904762,0.018933,1.070922,0.055356,4,2,2,36,1.234705,-5.954954,-2.388511,84,176,4.190476,6.119048,0.025244,2.035211,0.148802,5,3,5,118,7.189189,-8.309880,-7.673469,248,14.588235,98,131,2.673469,2.755102,0.013781,9.351047,0.010753,3,97,26,1,-1.049462,3.056280,7.437749,98,655,13.367347,16.448980,0.137808,2.475910,0.757773,23,5
2,Z1_H1_journaling,Z1_H1,journaling,17,41,43,2.097561,4.634146,0.026220,1.475000,0.003836,3,0.953488,2,2,1,-1.062701,-1.906504,-0.505550,55,113,4.109091,6.218182,0.038047,1.702830,0.199085,4,1,4,58,6.155766,-7.071880,-5.932385,55,156,5.672727,9.672727,0.052525,1.885343,0.202922,5,13,3,125,5.790476,-11.276194,-11.996126,219,12.882353,66,113,3.424242,4.090909,0.026340,4.328284,0.072753,4,65,11,17,1.154894,-1.513560,3.675586,65,363,11.169231,21.630769,0.174519,1.901740,0.510935,13,3
3,Z4_H5_journaling,Z4_H5,journaling,9,23,23,2.000000,2.608696,0.045455,1.000000,0.000000,2,0.958333,1,1,0,-1.040235,-2.307220,-0.816822,30,49,3.266667,4.066667,0.056322,1.092593,0.123967,3,1,2,10,0.553072,-3.860945,-1.900926,30,72,4.800000,6.066667,0.082759,1.716000,0.211823,4,5,2,43,1.787916,-6.269044,-5.437402,95,10.555556,39,53,2.717949,2.871795,0.035762,4.616915,0.051724,3,37,11,4,-0.332593,1.158426,5.515412,39,211,10.820513,14.358974,0.284750,1.716599,0.768823,15,2
4,Z16_H19_journaling,Z16_H19,journaling,11,7,7,2.000000,2.857143,0.166667,1.000000,0.000000,2,0.875000,1,1,0,NaN,-1.614895,-0.773277,17,19,2.235294,2.705882,0.069853,1.000000,0.109091,3,1,1,2,0.028074,-2.217518,-0.921997,17,26,3.058824,3.882353,0.095588,1.642857,0.230769,4,2,2,9,0.955116,-2.001437,-1.223905,59,5.363636,30,39,2.600000,2.666667,0.044828,5.158430,0.066667,3,23,13,2,-0.253959,2.715686,4.335952,29,66,4.551724,5.310345,0.162562,1.615385,0.612903,7,2


In [8]:
integrated_graphs.to_csv('/content/drive/MyDrive/Northwell Pipelines/Youtube Pipeline/9_taskwise_graphs/taskwise_graph_features_youtube.csv')